## Sentiment analysis using CNN

2. Load training dataset. 


In [2]:
%matplotlib inline
import xml.etree.ElementTree as ET
import utils
from utils import *
from keras.datasets import imdb
import numpy as np
import os

In [3]:
current_dir = os.getcwd()
DATA_DIR = current_dir + "/sentiments/data"
ORIGINAL_DATA = DATA_DIR + "/original"
TRAIN_DATA = DATA_DIR + "/train"
NEG_REVIEWS_TRAIN_FILE = TRAIN_DATA + "/negetivereviews.txt"
POS_REVIEWS_TRAIN_FILE = TRAIN_DATA + "/positivereviews.txt"
MODEL_PATH = current_dir + "/sentiments/models/"
WORDS_TO_IDX_DATA_FILE = DATA_DIR + "/words_to_idx.p"
IDX_TO_WORD_DATA_FILE = DATA_DIR + "/idx_to_words.p"
print DATA_DIR, ORIGINAL_DATA, TRAIN_DATA
STR_MAX_LEN = 500
vocabsize = 50000
MODEL_NAME = "model-s"+str(vocabsize)+"-v06-06-28"+".h5"
print MODEL_NAME

/home/ubuntu/nbs/sentiments/data /home/ubuntu/nbs/sentiments/data/original /home/ubuntu/nbs/sentiments/data/train
model-s50000-v06-06-28.h5


## Setup - One time 

review file format

      <review>
            <unique_id>B00005UKBG:bad:j._brodeur_"disgusted_consumer"</unique_id>
            <asin>B00005UKBG</asin>
            <product_name>Atlantic 1316 CD Storage Case (110-Capacity, Wave): Electronics</product_name>
            <product_type>electronics</product_type>
            <helpful>15 of 16</helpful>
            <rating>2.0</rating>
            <title>bad</title>
            <date>May 4, 2005</date>
            <reviewer>J. Brodeur "disgusted consumer"</reviewer>
            <reviewer_location>	</reviewer_location>
            <review_text>cons tips extremely easy on carpet and if you have a lot of cds stacked at the	top</review_text>
       </review>

1. Prepare training dataset from reviews folder
        a.Read the positive reviews xml and copy the first top 1000 into an array
        b.Read the negetive reviews xml and copy the first top 1000 into the array
        c.Save the array into file

In [ ]:
# read negetive reviews
tree = ET.parse(ORIGINAL_DATA+"/negative.review.xml")
root = tree.getroot()
negetiveTexts = [txt.text for txt in root.iter('review_text')]

#Open file 
negetiveReviewsFile = open(NEG_REVIEWS_TRAIN_FILE,"w+")
negetiveReviewsFile.write("\n".join(negetiveTexts[:1000]).encode('utf-8').strip())

# read positive reviews
#Open file 
print "reading file = "+ ORIGINAL_DATA+"/positive.review.xml"
tree = ET.parse(ORIGINAL_DATA+"/positive.review.xml")
root = tree.getroot()
positiveTexts = [txt.text for txt in root.iter('review_text')]

#writing to file 
print "writing  file = "+ POS_REVIEWS_TRAIN_FILE
positiveReviewsFile = open(POS_REVIEWS_TRAIN_FILE,"w+")
positiveReviewsFile.write("\n".join(positiveTexts[:1000]).encode('utf-8').strip())

## Step 1
1. Load imdb reviews words list from keras imdb db
2. Prepare an idx to word map.
3. sort the words to get top most used words


In [ ]:
## Load this one time and dump into a file for reuse
imdbwords = imdb.get_word_index();
pickle.dump(imdbwords, open(WORDS_TO_IDX_DATA_FILE, "wb" ))

In [4]:
widx = pickle.load(open( WORDS_TO_IDX_DATA_FILE, "rb" ))
idx2word = {v:k for k,v in widx.iteritems()}
print widx["fawn"]
print idx2word[34701]
print len(widx)

34701
fawn
88584


Sort the word index so that we can get the top 5000 words

In [5]:
widx_sorted_list = sorted(widx, key=widx.get)
print widx_sorted_list[:100]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more', 'when', 'very', 'up', 'no', 'time', 'she', 'even', 'my', 'would', 'which', 'only', 'story', 'really', 'see', 'their', 'had', 'can', 'were', 'me', 'well', 'than', 'we', 'much', 'been', 'bad', 'get', 'will', 'do', 'also', 'into', 'people', 'other', 'first', 'great', 'because', 'how', 'him', 'most', "don't", 'made', 'its', 'then', 'way', 'make', 'them', 'too', 'could', 'any', 'movies', 'after']


## Step 2
1. download the english training dataset and labels from s3

In [6]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
file = open(path, 'rb')

(train_data, train_labels), (test_data, test_labels) = pickle.load(file)

In [7]:
print(train_labels[0:1])
print(train_data[0:1])

[1]
[[23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215]]


In [ ]:
" ".join(idx2word[v] for v in train_data[0])

In [8]:
trnData = [np.array([i if i < vocabsize -1 else vocabsize -1 for i in s]) for s in train_data]
testData = [np.array([i if i < vocabsize -1 else vocabsize -1 for i in s]) for s in test_data]


In [11]:
print trnData[0]
print train_labels[0]

[23022   309     6     3  1069   209     9  2175    30     1   169    55    14    46    82  5869
    41   393   110   138    14  5359    58  4477   150     8     1  5032  5948   482    69     5
   261    12 23022 49999  2003     6    73  2436     5   632    71     6  5359     1 25279     5
  2004 10471     1  5941  1534    34    67    64   205   140    65  1232 49999 21145     1 49265
     4     1   223   901    29  3024    69     4     1  5863    10   694     2    65  1534    51
    10   216     1   387     8    60     3  1472  3724   802     5  3521   177     1   393    10
  1238 14030    30   309     3   353   344  2989   143   130     5  7804    28     4   126  5359
  1472  2375     5 23022   309    10   532    12   108  1470     4    58   556   101    12 23022
   309     6   227  4187    48     3  2237    12     9   215]
1


In [ ]:
lenstats = np.array(map(len, trnData))
lenstats.max(), lenstats.min(), lenstats.mean()

    To make a perfect matrix , pad the array with zeros

In [18]:
padTrnData = sequence.pad_sequences(trnData, maxlen=STR_MAX_LEN, value=0)
padTestData = sequence.pad_sequences(testData, maxlen=STR_MAX_LEN, value=0)
print padTrnData[0]
print padTestData.shape
print STR_MAX_LEN
print vocabsize

[    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0

## Build the CNN
1. build the model 
2. Train the model 
3. save the model.

In [12]:
model = Sequential()
model.add(Embedding(input_dim=vocabsize, output_dim=32, input_length=STR_MAX_LEN, dropout=0.2))
model.add(Dropout(0.2))
model.add(Convolution1D(64, 5, activation='relu', border_mode='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer=Adam(),  metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       1600000     embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500, 32)       0           embedding_1[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 500, 64)       10304       dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 500, 64)       0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [ ]:
model.fit(padTrnData,train_labels,validation_data=(padTestData, test_labels), nb_epoch=4, batch_size=64)

In [ ]:
model.save_weights(MODEL_PATH + MODEL_NAME)

## Prediction
1. take a text 
2. split it into array

In [ ]:
#text = "dont know what could have saved limp dispiriting yam but it definitely wasnt a lukewarm mushroom as murky and appealing as bong water"
#text = "Oh and to top it all off the staff was taking photo in front of the food so we couldn't get in and the manager was five feet away not even paying attention "
#text = "When I made the reservation they made sure to ask when and where our show was to ensure we had enough time to dine and still make it to our show"
#text = "Very nice little place. Prices are reasonable and food was good. Staff is courteous and patient with a non Spanish speaking guy...lol."
#text = "food was very good , but the service was amazing"
text="the director in that company is not good"

#text="I ordered 2 lb bag of the Canada Green grass seed. It germinated but then died within three weeks. They said that there was an unconditional money back or replacement guarantee on the product. I have contacted the company through their Best Buy Direct distributor and was told that someone front he... company would be contacting me. It has been here weeks and still nothing. I would like to file a complaint for the lack of response and the replacement refund guarantee."

#text = "The parents of two of the seven cousins killed in that crash have sued the truck's driver and the trucking company"

In [ ]:
model.load_weights(MODEL_PATH + MODEL_NAME)

In [ ]:
#text_clean = re.sub('\W+', ' ', text)
textWordsArray = np.array(text.lower().split())
print len(textWordsArray)
#textWordsArray
textWordsIdx = []
#textWordsIdxArray = [np.array(map(widx[i],(i if vocabsize -1 > i else vocabsize-1 for i in textWordsArray)))]

for w in textWordsArray:
    word = ''.join(c for c in w if (c.isalnum() or c == "'"))
    if word not in widx:
        textWordsIdx.append(vocabsize - 1)
        print " not found word = "+word
    elif widx[word] > vocabsize -1:
        textWordsIdx.append(vocabsize - 1)
        print "rare word = "+ word
    else:
        textWordsIdx.append(widx[word])

textWordsIdxArray = [np.array(textWordsIdx)]
#textWordsIdxArray = [np.array([i if i < vocabsize -1 else vocabsize -1 for i in s]) for s in textWordsIdxArray]
#print textWordsIdx

#print textWordsIdxArray
textIdxArrayPadded = sequence.pad_sequences(textWordsIdxArray,maxlen=STR_MAX_LEN, value=0)
#print textIdxArrayPadded
prediction = model.predict(textIdxArrayPadded, batch_size=1,verbose=1)
print prediction
if prediction[0][0] > 0.60:
    print "POSITIVE"
elif prediction[0][0] < 0.50:
    print "NEGETIVE"
else:
    print "NEUTRAL"